<a href="https://colab.research.google.com/github/Rob174/Astronomy/blob/Astronomy/Mod%C3%A8le000001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fonctions de base

In [0]:

##Python / Colab
from google.colab import files
from google.colab import drive
import os
from IPython.display import Image as imgIPython
from IPython.display import clear_output,display
import IPython
## Tensorflow keras
%tensorflow_version 1.x
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.python import debug as tf_debug
from tensorflow.python.client import device_lib

from tensorflow.keras.models import Sequential

from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense,Conv2D,Convolution2D,Activation,Conv2DTranspose
from tensorflow.keras.layers import MaxPooling2D,AveragePooling2D
from tensorflow.keras.layers import Dropout,Reshape,BatchNormalization
from tensorflow.keras.layers import concatenate,Concatenate,Subtract,Multiply,Average,Add
from tensorflow.keras.layers import UpSampling2D, Reshape,Flatten
from tensorflow.keras.layers import Lambda

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras import models
from tensorflow.keras.models import Model
import tensorflow.keras.losses
## Math libraries
import numpy as np
import scipy
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
##Images
from PIL import Image
import cv2
## Graph
from graphviz import render
from graphviz import Digraph,Graph

In [0]:
drive.mount('/content/drive')
%cd '/content/drive/My Drive/TIPE'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/TIPE


In [0]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/imports')
sys.path.append('/content/drive/My Drive/Colab Notebooks/imports/Tensorflow')
sys.path.append('/content/drive/My Drive/Colab Notebooks/imports/Graph_visualization')
## Custom library
from input_setup import *
from custom_layers import * 
from graph_global import * 
from graph_graphviz import * 
from graph_keras import * 

## Test fonction bruitage

In [0]:
max_step=200
max_it=2000
iteration = 100
train_data,validation_data,test_data = create_datasets()
# image,imageBruitee = next_batch_bruit_voile_2(5,train_data,256,np.float32,
#                                                             [[0,1.-iteration/(max_it+1)],[0,1.-iteration/(max_it+1)],[0,1.-iteration/(max_it+1)]],
#                                                             [iteration//100*100/max_it/2,iteration//100*100/max_it/2],[0,1])
taux_attenuation_max_log = 0.4#@param {type:'slider',min:0,max:5,step:0.1}
taux_attenuation_min_log = 1.3#@param {type:'slider',min:0,max:5,step:0.1}
ordre_gdr_bruit = 1.779#@param {type:'slider',min:0,max:4,step:0.001}
taux_bruit = 10**-ordre_gdr_bruit
plage_val = [taux_attenuation_min_log,taux_attenuation_max_log]
print("Facteur voile entre %f et %f"%(plage_val[0],plage_val[1]))
image,imageBruitee = next_batch_bruit_voile_2(5,train_data,256,np.float32,
                                                            [plage_val,plage_val,plage_val],
                                                            [taux_bruit,taux_bruit],[0,1])


plt.figure(figsize=(30,15))
it = 1
for statut in range(2):
    for img in range(5):
        plt.subplot(2,5,it)
        plt.imshow(image[img,:,:,:] if statut == 0 else imageBruitee[img,:,:,:])
        it += 1

NameError: ignored

# Conversions

In [0]:
def LarrayFloatToUint(L):
    return [np.array(array,np.uint) for array in L]

In [0]:
def convertToUintL(L):
    Lresult = []
    print("Entree : ",len(L))
    for i in range(len(L)):
        Lresult.append(np.array(normalisation(L[i],[0,1],[0,255]),dtype=np.uint8))
    print("Sortie : ",len(Lresult))
    return Lresult

In [0]:
def convertToUint(array):
    return np.array(normalisation(array,[0,1],[0,255]),dtype=np.uint8)

In [0]:
# cf https://stackoverflow.com/questions/43784921/how-to-display-custom-images-in-tensorboard-using-keras?noredirect=1#comment85726690_43784921
def make_image(tensor):
    """
    Convert an numpy representation image to Image protobuf.
    Copied from https://github.com/lanpa/tensorboard-pytorch/
    """
    from PIL import Image
    tensor = np.stack((tensor,tensor,tensor),axis=-1)
    height, width, channel = tensor.shape # numpy.ndarray
    image = Image.fromarray(tensor)
    import io
    output = io.BytesIO()
    image.save(output, format='PNG')
    image_string = output.getvalue()
    output.close()
    CHANNEL = 1
    var = tf.Summary.Image(height=height,
                         width=width,
                         colorspace=CHANNEL,
                         encoded_image_string=image_string)
    print("var : ",var)
    return var

In [0]:
def print_shape(x):
    print("Shape : ",x.get_shape().as_list())

In [0]:
def tauxApprentissage(epoch,ampl,tau,lim):
    taux = ampl*10**-((epoch)/tau)
    return taux if taux > lim else lim

In [0]:
def differenceAcceptee(epoch,ampl,tau,lim):
    taux = ampl*10**-((epoch)/tau)
    return taux if taux > lim else lim

In [0]:
def save_img(img_array_normalized,path):
    assert type(img_array_normalized) == np.ndarray, "Pass a numpy array"
    assert len(img_array_normalized.shape) == 4, "Pass a tensor, a batch of images with 4 dimensions"
    assert img_array_normalized.shape[-1] == 3, "Pass a tensor with 3 channels at the end to build rgb images"
    assert np.max(img_array_normalized) <=1, "Max tensor value must be 1 no %f"%np.max(img_array_normalized)
    individual_path = path
    for i in range(img_array_normalized.shape[0]):
        individual_path = path + "_batchIndex_%d"%(i)
        cv2.imwrite(path+'.jpg', np.uint8(img_array_normalized[i,:,:,:]*255))

## Layers graph mode implementation

### Graph, Keras

In [0]:
def rgb_to_magn_angle(x,draw=True):
    x,graph,index,improvements = x[0],x[1],x[2],x[3]
    x = Lambda(lambda x:K.cast(x,"complex64"))(x)
    x_list_magn = []
    x_list_angle = []
    for i in range(3):
        fft = Lambda(lambda x: tf.signal.fft2d(x[:,:,:,i]), output_shape=(None,199,199))(x)
        x_list_magn.append(Lambda(lambda fft:K.expand_dims(tf.math.abs(fft),axis=-1), output_shape=(None,199,199))(fft))
        x_list_angle.append(Lambda(lambda fft: K.expand_dims(tf.math.angle(fft),axis=-1), output_shape=(None,199,199))(fft))
    magn = Concatenate()(x_list_magn)
    angle = Concatenate()(x_list_angle)
    magn = Lambda(lambda magn: K.cast(magn,dtype=tf.float32), output_shape=(None,199,199))(magn)
    angle = Lambda(lambda angle: K.cast(angle,dtype=tf.float32), output_shape=(None,199,199))(angle)
    if draw == True:
#         print("Draw FFT at index %s"%g_get_current_id())
        graph.node(g_get_current_id(),"FFT")
        g_link(graph,index,g_get_current_id())
    global index_couches
    index_couches += 1
    return [magn,graph,g_get_past_id(),improvements],[angle,graph,g_get_past_id(),improvements]
def to_rgb(magn,angle,magn_train,angle_train,draw=True):
    magn,graph,magn_index,improvements = extract_inpt(magn)
    angle,graph,angle_index,improvements = extract_inpt(angle)
    magn = Lambda(lambda magn: K.cast(magn,dtype=tf.complex64))(magn)
    angle = Lambda(lambda angle:K.cast(angle,dtype=tf.complex64))(angle)
    x_list_canal = []
    for i in range(3):
        complx_tensor = Lambda(lambda mang_angle: mang_angle[0][:,:,:,i]*tf.exp(1j*mang_angle[1][:,:,:,i]))([magn,angle])
        ifft = Lambda(tf.signal.ifft2d)(complx_tensor)
        x_list_canal.append(K.expand_dims(ifft,axis=-1))
    images = Concatenate()(x_list_canal)
    images = K.cast(images,dtype=tf.float32)
    if draw == True:
        graph.node(g_get_current_id(),"RGB")
        if magn_train == True:
            g_link(graph,magn_index,g_get_current_id())
        if angle_train == True:
            g_link(graph,angle_index,g_get_current_id())
    global index_couches
    index_couches += 1
    return [images,graph,g_get_past_id(),improvements]

###Couches composites

In [0]:
def unit(inpt,k,f,dropout_r=None,activation=False,draw=False,init_index=None,identifier=False):
    if identifier == True:
        print("Identifier : ",identifier)
    if init_index != None:
        global index_couches
        index_couches = init_index
    couche = conv(inpt,k=k,f=f,s=1,draw=draw,identifier=identifier)
    couche = b_norm(couche,draw=draw,identifier=identifier)
    couche = lrn(couche,n=21,k=2,a=10**-4,b=0.75,draw=draw,identifier=identifier)
    if activation == True:
        couche = activ(couche,act_type="SELU",draw=draw,identifier=identifier)
    if dropout_r != None:
        couche = dropout(couche,r=dropout_r,draw=draw,identifier=identifier)
    return couche

In [0]:
def compare_shape(t1,t2):
    if len(t1) != len(t2):
        return False
    for elem1,elem2 in zip(t1,t2):
        if elem1 != elem2:
            return False
    return True

In [0]:
#Because we need to have both the former layer and the layer resulting of the next floor we have to split the "etage" in an down function and an up function that will create respectively the left and the right part 
def etage_down(inpt_before,pool_k,k,f,dropout_r=None,activation=False,draw=False,init_index=None):
    """We consider that a treatment has been already done on the inpt_before layer ;
    on the other side the etage function will restitute a layer with dimensions that match with inpt_before ;
    if we are at the last etage we will only call one / multiple units with the two ouputs layers
    In order to keep mostly the same weights we will always keep max pool and deconv layers in the process with deconv non trainable --> in the corresponding layers"""
    reduced = max_p(inpt_before,pool_k,draw=draw)
    after_process = unit(reduced,k,f,dropout_r=dropout_r,activation=True,draw=draw,init_index=init_index)
    return after_process

def etage_up(inpt_before,inpt_after,after_process,k,f,dropout_r=None,activation=False,draw=False,init_index=None):
    _,_,_,improvements_size_modif = extract_inpt(inpt_before)
    _,_,_,improvements_processing = extract_inpt(after_process)
    
    inpt_after[3] = improvements_processing
    after_process[3] = improvements_processing
    inpt_after = concat([inpt_after,after_process],operation=False, identifier=False,draw=draw)
    inpt_after_processed = unit(inpt_after,k,f,dropout_r=dropout_r,activation=activation,draw=draw,init_index=init_index)
    
    output = deconv(inpt_after_processed,k=2,f=inpt_before[0].get_shape().as_list()[-1],strides=(2,2),
                    tmp=False,draw=draw)
    if compare_shape(output[0].get_shape().as_list(),inpt_before[0].get_shape().as_list()) != True:
        raise Exception("No appropriate deconvolution kernel found with desired shape %s\nand input of deconvolution %s"%(inpt_before[0].get_shape().as_list(),output[0].get_shape().as_list()))
    return output

# Model

## Global model

In [0]:
def inception(inpt,draw=True):
    orig_graph, inpt = begin_cluster(past_couche=inpt,name="Inception%s"%(g_get_current_id()),color='red')
    
    T0 = unit(inpt,k=1,f=100,activation=True,dropout_r=0.5,draw=draw)
    T1 = unit(inpt,k=1,f=100,activation=True,dropout_r=0.5,draw=draw)
    T1 = unit(T1,k=1,f=100,activation=True,dropout_r=0.5,draw=draw)

    couche = concat([T0,T1],draw=draw)
    couche = conv(couche,k=1,f=100,s=1,draw=draw)
    couche = add([couche,inpt],draw=draw)
    end_cluster(last_couche=couche, orig_graph=orig_graph)
    return couche

In [0]:
def sauvegardeModele(entree_pure,entree_deterioree,model,iteration_entrainement,summary_writer,batch_size=5):
    global Llayers
    for canal_image in range(3):
        summary_image = tf.Summary(value=[tf.Summary.Value(tag="input_pure_canal_"+str(canal_image), 
                                image=make_image(np.array(entree_pure[0][0,:,:,canal_image]*255,dtype=np.uint8)))])
        summary_writer.add_summary(summary_image,iteration_entrainement)
        summary_image = tf.Summary(value=[tf.Summary.Value(tag="input_deterioree_canal_"+str(canal_image), 
                                image=make_image(np.array(entree_deterioree[0][0,:,:,canal_image]*255,dtype=np.uint8)))])
        summary_writer.add_summary(summary_image,iteration_entrainement)
        
    for p,entree in enumerate([entree_pure,entree_deterioree]):
        print("p value %d"%(p))
        layer_outputs,layer_names = [Llayers[i][0] for i in range(len(Llayers))],[Llayers[i][1] for i in range(len(Llayers))]
        model_calcul_image = Model(inputs=model.input,outputs=[model.output]+layer_outputs)
        sorties_couches = model_calcul_image.predict(entree, batch_size=batch_size)[1:] if len(layer_outputs) > 0 else [model_calcul_image.predict(entree, batch_size=batch_size)][1:]
        
        for index_couche,sortie_couche in enumerate(sorties_couches):
            layer_name = layer_names[index_couche]
            dim_sortie = sortie_couche.shape
            if len(dim_sortie) == 4:
                for canal_image in range(dim_sortie[-1]):
                    tag = layer_name
                    tag += 'pure' if p == 0 else 'deterioree'
                    tag += "_canal_" + str(canal_image)
                    summary_image = tf.Summary(value=[tf.Summary.Value(tag=tag, 
                                            image=make_image(np.array(sortie_couche[0,:,:,canal_image]*255,dtype=np.uint8)))])
                    summary_writer.add_summary(summary_image,iteration_entrainement)
    return None

In [0]:
from google.colab import output
def beep():
    output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')
    return None

In [0]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import time
continuer = None
def block_or_continue():
    beep()
    print("Would you resume training ? True or False ?")
    return bool(input())

In [0]:
def backup(image,imageBruitee,gen_disc,disc,index,summary_writer):
    x = [np.concatenate((image[0],imageBruitee[0])),np.concatenate((image[1],imageBruitee[1]))]
    sauvegardeModele(image,imageBruitee,gen_disc,index,summary_writer)
    p_gen_disc = gen_disc.predict(x)[0]
    p_disc = disc.predict(x)[0]

    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité gen-disc img 0 (orig pure) pure : ", 
                                        simple_value=p_gen_disc) ])
    summary_writer.add_summary(summary_loss,index)
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité disc img 0 (orig pure) pure : ", 
                                        simple_value=p_disc) ])
    summary_writer.add_summary(summary_loss,index)
    x_proba = [np.concatenate((imageBruitee[0],imageBruitee[0])),np.concatenate((imageBruitee[1],imageBruitee[1]))]
    p_gen_disc = gen_disc.predict(x_proba)[0]
    p_disc = disc.predict(x_proba)[0]
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité gen-disc img 0 (orig détériorée) pure : ", 
                                        simple_value=p_gen_disc) ])
    summary_writer.add_summary(summary_loss,index)
    summary_loss = tf.Summary(value=[tf.Summary.Value(tag="Probabilité disc img 0 (orig détériorée) pure : ", 
                                        simple_value=p_disc) ])
    summary_writer.add_summary(summary_loss,index)

In [0]:
def custom_loss(y_true,y_pred):
    return K.mean(K.square(y_pred - y_true),axis=-1)+K.max(K.square(y_pred - y_true),axis=-1)

In [0]:
def to_fft(img_tensor):
    L = []
    print(img_tensor.shape[0],img_tensor.get_shape().as_list())
    for index_image in range(img_tensor.get_shape().as_list()[0]):
        L.append(to_fft_single(img_tensor[index_image,:,:,:]))
    return np.stack(L,axis=0)

def to_img(fft_cplx):
    return to_uint8_array(np.fft.ifft2(fft_cplx))
def to_fft_single(img):
    fft_cplx = np.fft.fft2(img)
    fft = 20*np.log(np.abs(fft_cplx)+10**-5*np.ones(img.shape))
    fft = fft + np.abs(np.min(fft))*np.ones(fft.shape)
    fft = fft / (np.max(fft)*np.ones(fft.shape))*(255*np.ones(fft.shape))
    fft_uint8 = to_uint8_array(fft)
    print(np.max(fft_uint8),np.min(fft_uint8),fft_uint8.shape,fft_uint8.dtype)
    return fft_cplx

def to_uint8_array(array):
    return array.astype(np.uint8)
def next_batch_bruit_voile_fft():
    image,imageBruitee = next_batch_bruit_voile_2(10,images,199,np.float32,[1,1,1],[50,50])
    

## FFT Pretraining

### Model

In [0]:
def analyse_progressive_training_model(couche,step=0,init=2,draw=True):
    #input size has to be 2**4=16 min but we wil take 2**8=256
    level0_in = unit(couche,k=2,f=100,activation=True,dropout_r=0.1,init_index=init,draw=draw)#Output size around 256
    down1 = etage_down(level0_in,pool_k=2,k=2,f=100,dropout_r=0.1,activation=True,draw=draw)#Output size around 128
    down2 = etage_down(down1,pool_k=2,k=2,f=100,dropout_r=0.1,activation=True,draw=draw)#Output size around 64
    down3 = etage_down(down2,pool_k=2,k=2,f=100,dropout_r=0.1,activation=True,draw=draw)#Output size around 32
    down4 = etage_down(down3,pool_k=2,k=2,f=100,dropout_r=0.1,activation=True,draw=draw)#Output size around 16
    
    final_analysis = unit(down4,k=2,f=100,activation=True,draw=draw)#Output size around 16
    
    up4 = etage_up(inpt_before=down3,inpt_after=final_analysis,after_process=down4,k=2,f=100,dropout_r=0.1,activation=True,draw=draw)#Output size around 32
    up3 = etage_up(inpt_before=down2,inpt_after=up4,after_process=down3,k=2,f=100,dropout_r=0.1,activation=True,draw=draw)#Output size around 64
    up2 = etage_up(inpt_before=down1,inpt_after=up3,after_process=down2,k=2,f=100,dropout_r=0.1,activation=True,draw=draw)#Output size around 128
    up1 = etage_up(inpt_before=level0_in,inpt_after=up2,after_process=down1,k=2,f=100,dropout_r=0.1,activation=True,draw=draw)#Output size around 256
    level0_out = concat([level0_in,up1],draw=draw)
    level0_out[3] = couche[3]
    level0_out = unit(level0_out,k=2,f=3,activation=True,dropout_r=0.1,draw=draw)#Bonne taille
    return  level0_out


In [0]:
index_couches = -12
def model_fft_training(step,half_batch_size,name,magn_select=True,angle_select=True):#else angle
    global index_disc
    global index_couches
    index_couches = 1
    graph = new_graph()
    inpt = g_input(graph=graph, improvements=step, shape=(256, 256, 3),content_name="Image")
    inpt_real = g_input(graph=graph, improvements=step, shape=(256, 256, 3),content_name="Image_real",draw=False)

    magn,angle = rgb_to_magn_angle(inpt,draw=True)
    r_magn,r_angle = rgb_to_magn_angle(inpt_real,draw=False)

    if magn_select == True:
        magn = analyse_progressive_training_model(magn,step=step,init=3)
    if angle_select == True:
        angle = analyse_progressive_training_model(angle,step=step,init=150)
    img_rgb = to_rgb(magn=magn,angle=angle,magn_train=magn_select,angle_train=angle_select) 
    model = Model(inputs=[inpt[0],inpt_real[0]],outputs=img_rgb[0],name='gen')
    end_graph(img_rgb[1],name=name)
    return model, r_magn[0],r_angle[0], magn[0], angle[0]

In [0]:
# validation = None
# for step in range(38,100):#21
#     for training_L in [[False,True],[True,False],[True,True]]:
#         base_name = "Pretrain_test_fft_%d"%10
#         graph_name = base_name
#         if training_L[0] == True:
#             graph_name += "_magn"
#         if training_L[1] == True:
#             graph_name += "_angle"
#         graph_name = graph_name + "_%d"%(step)
#         backup_path = graph_name + ".h5"
#         global index_couches
#         model_fft_training(step,half_batch_size=5,name=graph_name,magn_select=training_L[0],angle_select=training_L[1])
#         print("Is it a correct graph for step %d angle %r et magn %r ?"%(step,training_L[0],training_L[1]))
#         print("path : %s"%('%s.png'%(graph_name)))
#         graph = imgIPython(filename='%s.png'%(graph_name)) 
#         display(graph)
#         validation = input()
#         if validation == "e":
#             break
#         if validation != "t":
#             raise Exception("The graph is not correct")
    
#     if validation == "e":
#         break

### Backup

In [0]:
def restore_modified_network(past_model,new_model):
    layers = [layer.name for layer in new_model.layers]
    for layer in past_model.layers:
        if layer.name in layers:
            print("Restoring layer %s"%layer.name)
            try:
                new_model.get_layer(layer.name).set_weights(layer.get_weights())
            except Exception as e:
                print("Can't load weights because ")
                print(str(e))
                print("So, nothing done for the layer %s"%(layer.name))

In [0]:
def search_restore_pt(index_model,model,magn_trained,angle_trained,base_name):
    best_angle_training = None
    best_magn_training = None
    max_step_angle = 0
    max_step_magn = 0
    folder = "Model%d/"%index_model
    if os.path.exists(folder) == False:
        print("No restore point found")
        return None
    for file in os.listdir(folder):
        if file.endswith(".h5") and base_name in file:
            if "magn" in file and int(file.split(".")[0].split("_")[-1]) >= max_step_magn:
                best_magn_training = folder + file
                max_step_magn = int(file.split(".")[0].split("_")[-1])
            if "angle" in file and int(file.split(".")[0].split("_")[-1]) >= max_step_angle:
                best_angle_training = folder + file
                max_step_angle = int(file.split(".")[0].split("_")[-1])
    # Pour cette étape d'entrainement, 
    #       - soit uniquement l'un des deux a été entrainé séparément : cas où à l'iteration step on arrive à magn = True et angle = True 
    #               -> magn déjà entrainé pour step mais angle en est encore à step-1
    #           --> on restaure d'abord le fichier contenant angle et magn de l'étape précédante puis celui d'entrainement particulier
    #           --> soit d'abord dans tous les cas le fichier de angle puis celui de magn
    #       - soit les deux ont été entrainés séparément
    #           --> on restaure les deux fichier sans ordre préférentiel car chaque fichier ne contient pas d'info sur un reseau analysant l'autre partie de l'image
    #       - soit les deux ont été entrainés en commun 
    #           --> on restaure d'abord le fichier contenant angle et magn de l'étape précédante puis celui d'entrainement particulier
    # Dans l'ordre on fait angle1 -> magn1 -> magn_angle1 -> angle2 -> magn2 -> magn_angle2 -> ...
    if (angle_trained == True and magn_trained == False) or (angle_trained == False and magn_trained == True) and (best_angle_training != None and best_magn_training != None):
        # Cas angle2 ou magn2
        print("Restoring weights...")
        model_prec,_,_, _,_ = model_fft_training(step=max_step_magn,#To avoid problems after training angle of this step
                                                                           half_batch_size=5,
                                                                  name=graph_name,magn_select=True,
                                                                  angle_select=True)
        model_prec.load_weights(best_angle_training,by_name=True)
        restore_modified_network(past_model=model_prec,new_model=model)
    elif (angle_trained == True and magn_trained == True) and (best_angle_training != None and best_magn_training != None):
        print("Restoring weights from models angle_magn into magn_angle...")
        # Cas magn_angle1 : on construit angle1 et magn1 et on met les variables dans le nouveau modèle
        model_prec_angle,_,_, _,_ = model_fft_training(step=max_step_magn,half_batch_size=5,
                                                                  name=base_name,magn_select=True,
                                                                  angle_select=True)
        model_prec_magn,_,_, _,_ = model_fft_training(step=max_step_magn,half_batch_size=5,
                                                                  name=base_name,magn_select=True,
                                                                  angle_select=True)
        model_prec_angle.load_weights(best_angle_training,by_name=True)
        restore_modified_network(past_model=model_prec_angle,new_model=model)
        model_prec_magn.load_weights(best_magn_training,by_name=True)
        restore_modified_network(past_model=model_prec_magn,new_model=model)
    else:
        print("%s restore point found for angle and %s restore point found for magn"%("No" if best_angle_training == None else "One","no" if best_magn_training == None else "one"))

In [0]:
def backup_fft(num_model,model,image,imageBruitee,graph_name,step,iteration,metrics_list_names,model_state_backup_int=5):
    if os.path.exists("Model%d"%num_model) == False:
        os.mkdir("Model%d"%num_model)
    folder = "Model%d/"%num_model
    metrics_list_values_clean = model.test_on_batch([image,image],image)
    metrics_list_values_noise = model.test_on_batch([imageBruitee,image],image)
    if len(metrics_list_values_clean) != len(metrics_list_names) and len(metrics_list_values_noise) != len(metrics_list_names):
        raise Exception("Metrics names number and model outputs not matching with %d values for model output and %d values for names"%(len(metrics_list_values_clean),len(metrics_list_names)))
    metrics_names = []
    metrics_list_values = []
    for i,name in enumerate(metrics_list_names):
        metrics_names += [name+"_clean",name+"_noise"]
        metrics_list_values += [metrics_list_values_clean[i],metrics_list_values_noise[i]]
    metrics_names = ["Iteration"] + metrics_names
    metrics_list_values = [iteration] + metrics_list_values
    if iteration % model_state_backup_int == 0:
        model.save_weights(folder + graph_name+".h5")
        save_img(image,folder + graph_name+"_%d_img"%iteration)
        save_img(imageBruitee,folder + graph_name+"_%d_img_bruit"%iteration)
        img_gen = model.predict([image,image])
        img_bruit_gen = model.predict([imageBruitee,image])
        save_img(np.clip(img_gen,0,1),folder + graph_name+"_%d_img_gen"%iteration)
        save_img(np.clip(img_bruit_gen,0,1),folder + graph_name+"_%d_img_bruit_gen"%iteration)

    append_write = "w"
    backup_file_path = folder + graph_name+"_step%d_metrics.txt"%step
    if os.path.exists(backup_file_path):
        append_write = 'a' # append if already exists
    with open(backup_file_path,append_write) as file:
        if os.stat(backup_file_path).st_size == 0:
            file.write(",".join(list(map(lambda x:str(x),metrics_names)))+"\n")
        file.write(",".join(list(map(lambda x:str(x),metrics_list_values)))+"\n")
    print("".join([metrics_names[i] + " : " + str(metrics_list_values[i]) + "; " for i in range(len(metrics_list_values))]))


### Loss & Metrics

In [0]:
def fft_loss(r_magn,r_angle, gen_magn,gen_angle):
    def loss(y_true,y_pred):
        return K.mean(K.square(r_magn - gen_magn), axis=-1) + K.mean(K.square(r_angle - gen_angle), axis=-1)
    return loss

In [0]:
Lmetrics = []
Lnames = []
def generate_functions(name,type_data,input_first_function,expression):
    global Lmetrics
    global Lnames
    Lnames.append("%s"%(expression).replace("y","rgb"))
    Lmetrics.append("metrics_%s_%s(%s)"%(type_data,name,",".join(input_first_function)))
    print("def metrics_%s_%s(%s):\n\tdef %s_%s(y_true,y_pred):\n\t\treturn %s\n\treturn %s_%s"%(type_data,name,",".join(input_first_function),type_data,name,expression,type_data,name))
    print()
for minmax in ["min","max"]:
    for magnangle in ["magn","angle"]:
        generate_functions("%s_%s"%(minmax,magnangle),"fft",["r_%s"%(magnangle)],"K.%s(r_%s)"%(minmax,magnangle))
for magnangle in ["magn","angle"]:
    generate_functions("max_abs_diff_%s"%(magnangle),"fft",["r_%s"%(magnangle),"gen_%s"%(magnangle)],"K.max(K.abs(r_%s-gen_%s))"%(magnangle,magnangle))
for magnangle in ["magn","angle"]:
    generate_functions("mean_abs_diff_%s"%(magnangle),"fft",["r_%s"%(magnangle),"gen_%s"%(magnangle)],"K.mean(K.abs(r_%s-gen_%s))"%(magnangle,magnangle))
for magnangle in ["magn","angle"]:
    generate_functions("var_abs_diff_%s"%(magnangle),"fft",["r_%s"%(magnangle),"gen_%s"%(magnangle)],"K.var(K.abs(r_%s-gen_%s))"%(magnangle,magnangle))
generate_functions("max_abs_diff_%s"%(magnangle),"rgb",[],"K.mean(K.abs(y_true-y_pred))")
print("[%s]"%(",".join(Lmetrics)))
print("[%s]"%(",".join(list(map(lambda x:"'"+x+"'",Lnames)))))

In [0]:
def metrics_fft_min_magn(r_magn):
	def fft_min_magn(y_true,y_pred):
		return K.min(r_magn)
	return fft_min_magn

def metrics_fft_min_angle(r_angle):
	def fft_min_angle(y_true,y_pred):
		return K.min(r_angle)
	return fft_min_angle

def metrics_fft_max_magn(r_magn):
	def fft_max_magn(y_true,y_pred):
		return K.max(r_magn)
	return fft_max_magn

def metrics_fft_max_angle(r_angle):
	def fft_max_angle(y_true,y_pred):
		return K.max(r_angle)
	return fft_max_angle

def metrics_fft_max_abs_diff_magn(r_magn,gen_magn):
	def fft_max_abs_diff_magn(y_true,y_pred):
		return K.max(K.abs(r_magn-gen_magn))
	return fft_max_abs_diff_magn

def metrics_fft_max_abs_diff_angle(r_angle,gen_angle):
	def fft_max_abs_diff_angle(y_true,y_pred):
		return K.max(K.abs(r_angle-gen_angle))
	return fft_max_abs_diff_angle

def metrics_fft_mean_abs_diff_magn(r_magn,gen_magn):
	def fft_mean_abs_diff_magn(y_true,y_pred):
		return K.mean(K.abs(r_magn-gen_magn))
	return fft_mean_abs_diff_magn

def metrics_fft_mean_abs_diff_angle(r_angle,gen_angle):
	def fft_mean_abs_diff_angle(y_true,y_pred):
		return K.mean(K.abs(r_angle-gen_angle))
	return fft_mean_abs_diff_angle

def metrics_fft_var_abs_diff_magn(r_magn,gen_magn):
	def fft_var_abs_diff_magn(y_true,y_pred):
		return K.var(K.abs(r_magn-gen_magn))
	return fft_var_abs_diff_magn

def metrics_fft_var_abs_diff_angle(r_angle,gen_angle):
	def fft_var_abs_diff_angle(y_true,y_pred):
		return K.var(K.abs(r_angle-gen_angle))
	return fft_var_abs_diff_angle

def metrics_rgb_max_abs_diff_angle():
	def rgb_max_abs_diff_angle(y_true,y_pred):
		return K.mean(K.abs(y_true-y_pred))
	return rgb_max_abs_diff_angle

### Training

In [0]:
def train_progressive_model(index_model,max_step=200, max_it=2000):
    train_data,validation_data,test_data = create_datasets()
    max_total_it = (max_step-1)*3*max_it
    tot_iteration = 0
    for step in range(40,max_step):
        magn_choice, angle_choice = True,True
        if magn_choice == False and angle_choice == False:
            continue

        base_name = "Pretrain_fft_%d"%index_model
        graph_name = base_name
        if magn_choice == True:
            graph_name += "_magn"
        if angle_choice == True:
            graph_name += "_angle"
        graph_name = graph_name + "_%d"%(step)
        backup_path = graph_name + ".h5"
        model,r_magn,r_angle, gen_magn,gen_angle = model_fft_training(step=step,half_batch_size=5,
                                                                        name=graph_name,magn_select=magn_choice,angle_select=angle_choice)
        
        metrics_list = [metrics_fft_min_magn(r_magn),metrics_fft_min_angle(r_angle),
                        metrics_fft_max_magn(r_magn),metrics_fft_max_angle(r_angle),
                        metrics_fft_max_abs_diff_magn(r_magn,gen_magn),metrics_fft_max_abs_diff_angle(r_angle,gen_angle),
                        metrics_fft_mean_abs_diff_magn(r_magn,gen_magn),metrics_fft_mean_abs_diff_angle(r_angle,gen_angle),
                        metrics_fft_var_abs_diff_magn(r_magn,gen_magn),metrics_fft_var_abs_diff_angle(r_angle,gen_angle),
                        metrics_rgb_max_abs_diff_angle()]
        metrics_list_names = ["Loss",'K.min(r_magn)','K.min(r_angle)',
                                'K.max(r_magn)','K.max(r_angle)',
                                'K.max(K.abs(r_magn-gen_magn))','K.max(K.abs(r_angle-gen_angle))',
                                'K.mean(K.abs(r_magn-gen_magn))','K.mean(K.abs(r_angle-gen_angle))',
                                'K.var(K.abs(r_magn-gen_magn))','K.var(K.abs(r_angle-gen_angle))',
                                'K.mean(K.abs(rgb_true-rgb_pred))']
        optimizer = Adam(learning_rate=1e-5, beta_1=0.9, beta_2=0.999, amsgrad=False)
        model.compile(loss=fft_loss(r_magn,r_angle, gen_magn,gen_angle),metrics=metrics_list,optimizer="Adam")
        search_restore_pt(index_model,model=model, magn_trained=magn_choice,angle_trained=angle_choice,
                            base_name=graph_name)
        for iteration in range(max_it):
            if step == 40 and iteration < 1406:
                continue
            image,imageBruitee = next_batch_bruit_voile_2(5,train_data,256,np.float32,
                                                            [[0,1.-iteration/(max_it+1)],[0,1.-iteration/(max_it+1)],[0,1.-iteration/(max_it+1)]],
                                                            [iteration//100*100/max_it/2,iteration//100*100/max_it/2],[0,1])
            x = np.concatenate((image,imageBruitee),axis=0)
            y = np.concatenate((image,image),axis=0)
            metrics_list_values = model.train_on_batch([x,y],y)
            loss = metrics_list_values[0]
            backup_fft(index_model,model,image,imageBruitee,graph_name,step,iteration,metrics_list_names,model_state_backup_int=5)
            whose_is_training = ""
            if magn_choice == True and angle_choice == True:
                whose_is_training += "magn et angle" 
            elif magn_choice == True:
                whose_is_training += "magn" 
            elif angle_choice == True:
                whose_is_training += "angle" 
            print("Etape d'entrainement %d de %s iteration %d statut %d pourcent, erreur %.5e"%(step,whose_is_training, iteration,tot_iteration/max_total_it,loss))
            tot_iteration += 1

In [0]:
# train_progressive_model(11)

# Discriminator pretraining


## Model

In [0]:
def generate_model():
    inpt = Input(shape=(256, 256, 3), name="Input")
    xception_model = tensorflow.keras.applications.xception.Xception(include_top=False, weights=None, input_tensor=inpt, input_shape=(256,256,3), pooling='max', classes=1)
    filterModification = Dense(units=1,activation=None)(xception_model.output)
    return Model(inputs=inpt,outputs=filterModification, name='disc')

## Losses & metrics

In [0]:
Lmetrics = []
Lnames = []
generate_functions("max_abs_diff_%s"%(magnangle),"rgb",[],"K.max(K.abs(y_true-y_pred))")
generate_functions("mean_abs_diff_%s"%(magnangle),"rgb",[],"K.mean(K.abs(y_true-y_pred))")
generate_functions("std_abs_diff_%s"%(magnangle),"rgb",[],"K.std(K.abs(y_true-y_pred))")
print("[%s]"%(",".join(Lmetrics)))
print("[%s]"%(",".join(list(map(lambda x:"'"+x+"'",Lnames)))))

In [0]:
def metrics_rgb_max_abs_diff_angle():
	def rgb_max_abs_diff_angle(y_true,y_pred):
		return K.max(K.abs(y_true-y_pred))
	return rgb_max_abs_diff_angle

def metrics_rgb_mean_abs_diff_angle():
	def rgb_mean_abs_diff_angle(y_true,y_pred):
		return K.mean(K.abs(y_true-y_pred))
	return rgb_mean_abs_diff_angle

def metrics_rgb_std_abs_diff_angle():
	def rgb_std_abs_diff_angle(y_true,y_pred):
		return K.std(K.abs(y_true-y_pred))
	return rgb_std_abs_diff_angle

## Backup

In [0]:
def search_restore_pt(index_model,model,base_name):
    """
    We only have to restore previous version of the model"""
    folder = "Model_disc%d/"%index_model
    if os.path.exists(folder) == False:
        print("No restore point found")
        return None
    model_path = ""
    for file in os.listdir(folder):
        if file.endswith(".h5") and base_name in file:
            model_path = file
    if model_path == "":
        print("No restore point found")
        return None
    try:
        model.load_weights(model_path,by_name=True)
    except Exception as e:
        print(str(e))
        print("Skipping restoration")

In [0]:
def append_or_write(backup_file_path):
    mode ="w"
    if os.path.exists(backup_file_path):
        append_write = 'a'
    return mode

In [0]:
def write_to_file(backup_file_path,titles,data_list_list):
    with open(backup_file_path,'a') as file:
        if os.stat(backup_file_path).st_size == 0:
            file.write(",".join(titles)+"\n")
        for line in data_list_list:
            file.write(",".join(line)+"\n")

In [0]:
def generate_input_output(img,imgNoise):
    """D'après la fonction backup_fft ci-dessus on prend comme convention que l'IA prédit la probabilité que 
    l'image d'entrée soit bruitée
    """
    inpt = np.concatenate((img,imgNoise),axis=0)
    outpt = np.zeros(shape=(inpt.shape[0]))
    return inpt,outpt

In [0]:
def backup_fft(num_model,model,image,imageBruitee,graph_name,step,iteration,metrics_list_names,model_state_backup_int=5):
    folder = "Model_disc%d/"%num_model
    if os.path.exists(folder) == False:
        os.mkdir(folder)
    
    outpt_clean = np.zeros(shape=(image.shape[0]))
    outpt_noise = np.zeros(shape=(imageBruitee.shape[0]))

    metrics_list_values_clean = model.test_on_batch(image,outpt_clean)
    metrics_list_values_noise = model.test_on_batch(imageBruitee,outpt_noise)
    if len(metrics_list_values_clean) != len(metrics_list_names) and len(metrics_list_values_noise) != len(metrics_list_names):
        print("metrics_list_values_clean : ",metrics_list_values_clean)
        print("metrics_list_names : ",metrics_list_names)
        raise Exception("Metrics names number and model outputs not matching with %d values for model output and %d values for names"%(len(metrics_list_values_clean),len(metrics_list_names)))
    metrics_names = []
    metrics_list_values = []
    for i,name in enumerate(metrics_list_names):
        metrics_names += [name+"_clean",name+"_noise"]
        metrics_list_values += [metrics_list_values_clean[i],metrics_list_values_noise[i]]
    metrics_names = ["Iteration"] + metrics_names
    metrics_list_values = [iteration] + metrics_list_values
    if iteration % model_state_backup_int == 0:
        model.save_weights(folder + graph_name+".h5")
        save_img(image,folder + graph_name+"_%d_img"%iteration)
        save_img(imageBruitee,folder + graph_name+"_%d_img_bruit"%iteration)
        prediction_clean_gen = model.predict(image)
        prediction_clean_bruit_gen = model.predict(imageBruitee)
        
        backup_prediction_path = folder + graph_name + "_step%d_predictions.txt"%(step)
        titles = ["Prediction_bruitee_orig_clean","Prediction_bruitee_orig_noise"]
        predictions = [str(prediction_clean_gen[0]),str(prediction_clean_bruit_gen[0])]
        write_to_file(backup_file_path=backup_prediction_path,titles=titles,data_list_list=[predictions])

    backup_file_path = folder + graph_name+"_step%d_metrics.txt"%step
    titles = list(map(lambda x:str(x),metrics_names))
    data = list(map(lambda x:str(x),metrics_list_values))
    write_to_file(backup_file_path=backup_file_path,titles=titles,data_list_list=[data])
    print("".join([metrics_names[i] + " : " + str(metrics_list_values[i]) + "; " for i in range(len(metrics_list_values))]))


## Training

In [0]:
def train_progressive_model(index_model,max_step=200, max_it=2000):
    train_data,validation_data,test_data = create_datasets()
    max_total_it = (max_step-1)*3*max_it
    tot_iteration = 0
    base_name = "Pretrain_fft_disc_%d"%index_model
    for step in range(max_step):
        graph_name = base_name
        graph_name = graph_name + "_%d"%(step)
        backup_path = graph_name + ".h5"
        model = generate_model()
            
        metrics_list = [metrics_rgb_max_abs_diff_angle(),metrics_rgb_mean_abs_diff_angle(),metrics_rgb_std_abs_diff_angle()]
        metrics_list_names = ['binary_crossentropy','K.max(K.abs(rgb_true-rgb_pred))','K.mean(K.abs(rgb_true-rgb_pred))','K.std(K.abs(rgb_true-rgb_pred))']

        model.compile(loss="binary_crossentropy",metrics=metrics_list,optimizer="Adam")
        search_restore_pt(index_model,model=model,
                            base_name=graph_name)
        for iteration in range(max_it):
            image,imageBruitee = next_batch_bruit_voile_2(5,train_data,256,np.float32,
                                                            [[0,1.-iteration/(max_it+1)],[0,1.-iteration/(max_it+1)],[0,1.-iteration/(max_it+1)]],
                                                            [iteration//100*100/max_it/2,iteration//100*100/max_it/2],[0,1])
            inpt,outpt = generate_input_output(image,imageBruitee)
            metrics_list_values = model.train_on_batch(inpt,outpt)
            loss = metrics_list_values[0]
            backup_fft(index_model,model,image,imageBruitee,graph_name,step,iteration,metrics_list_names,model_state_backup_int=5)
            
            print("Etape d'entrainement %d iteration %d statut %d pourcent, erreur %.5e"%(step, iteration,tot_iteration/max_total_it,loss))
            tot_iteration += 1

In [0]:
# train_progressive_model(index_model=11)

# GAN Pretrain FFT

##Model

In [0]:
index_couches = -12
def model_training(step,half_batch_size,name,magn_select=True,angle_select=True):#else angle
    global index_disc
    global index_couches
    index_couches = 1
    graph = new_graph()
    inpt = g_input(graph=graph, improvements=step, shape=(256, 256, 3),content_name="Image")
    output = analyse_progressive_training_model(inpt,step=step,init=3)
    model = Model(inputs=[inpt[0]],outputs=output[0],name='gen')
    end_graph(output[1],name=name)
    return model

In [0]:

K.clear_session()
def discriminateur():
    return generate_model()
def generate_models(name):
    gen_model = model_training(step=50,half_batch_size=5,name=name)
    gen = gen_model(gen_model.input)
    disc = discriminateur()
    disc_gan = disc(gen)
    inpt_gen_output = Input(shape=(256,256,3))
    return gen_model,disc,Model(inputs=[gen_model.input,inpt_gen_output],outputs=disc_gan),inpt_gen_output,gen

## Backup

In [0]:
def search_restore_pt(index_model,model_gen_disc,base_name):
    best_training = None
    max_step = 0
    folder = "Model_gan_fft11/"#%index_model
    # if os.path.exists(folder) == False:
    #     print("No restore point found")
    #     return None
    for file in os.listdir(folder):
        if file.endswith(".h5") and base_name in file:
            print("file : ",file,"gan" in file, int(file.split(".")[0].split("_")[-1]))
            if "gan" in file and int(file.split(".")[0].split("_")[-1]) >= max_step:
                best_training = folder + file
                max_step = int(file.split(".")[0].split("_")[-1])
    if (best_training == None):
        print("No GAN training found ; Restoring from splited files")
        pathDisc = "/content/drive/My Drive/TIPE/Model_disc11/Pretrain_fft_disc_11_4.h5"#@param {type:"string"}
        pathGen = "/content/drive/My Drive/TIPE/Model11/Pretrain_fft_11_magn_angle_43.h5"#@param {type:"string"}
        ## Restore gen part
        # model_prec_fft,_,_, _,_ = model_fft_training(step=43,half_batch_size=5,
        #                                                           name="Pretrain_fft_11_magn_angle_4",magn_select=True,
        #                                                           angle_select=True)
        # model_prec_fft.load_weights(pathGen,by_name=True)
        # restore_modified_network(past_model=model_prec_fft,new_model=model_gen_disc)
        ## Restore disc part
        model_disc_prec = generate_model()
        model_disc_prec.load_weights(pathDisc,by_name=True)
        restore_modified_network(past_model=model_disc_prec,new_model=model_gen_disc)

    elif (best_training != None):
        print("Restoring weights...")
        model_gen_disc.load_weights(best_training)
    

In [0]:
def backup_fft(num_model,model_gen,model_disc,model_gan,image,imageBruitee,graph_name,step,iteration,metrics_list_names_gen,metrics_list_names_disc,model_state_backup_int=5):
    
    folder = "Model_gan_fft%d/"%num_model
    if os.path.exists(folder) == False:
        os.mkdir(folder)
    metrics_list_names = ["Iterations"]
    metrics_list_values = [iteration]
    def format_metrics(current_metrics_list_names,metrics_list_values_clean,metrics_list_values_noise,metrics_list_names,metrics_list_values,identifiant=''):
        for i,name in enumerate(current_metrics_list_names):
            if identifiant != '':
                metrics_list_names += [name+"_%s_clean"%identifiant,name+"_%s_noise"%identifiant]
            else:
                metrics_list_names += [name+"_clean",name+"_noise"]
            metrics_list_values += [metrics_list_values_clean[i],metrics_list_values_noise[i]]
        return metrics_list_names,metrics_list_values
    metrics_list_values_clean = model_gen.test_on_batch(image,image)
    metrics_list_values_noise = model_gen.test_on_batch(imageBruitee,image)
    metrics_list_names, metrics_list_values = format_metrics(current_metrics_list_names=metrics_list_names_gen,metrics_list_values_clean=metrics_list_values_clean,metrics_list_values_noise=metrics_list_values_noise,metrics_list_names=metrics_list_names,metrics_list_values=metrics_list_values,identifiant='gen')

    input_gen_clean = model_gen.predict(image)
    input_gen_noise = model_gen.predict(imageBruitee)
    oupt_disc_clean = model_disc.predict(input_gen_clean)
    oupt_disc_noise = model_disc.predict(input_gen_noise)
    outpt_clean = np.zeros(shape=(image.shape[0]))
    outpt_noise = np.zeros(shape=(imageBruitee.shape[0]))
    metrics_list_values_clean = model_disc.test_on_batch(image,outpt_clean)
    metrics_list_values_noise = model_disc.test_on_batch(imageBruitee,outpt_noise)
    metrics_list_names, metrics_list_values = format_metrics(current_metrics_list_names=metrics_list_names_disc,metrics_list_values_clean=metrics_list_values_clean,metrics_list_values_noise=metrics_list_values_noise,metrics_list_names=metrics_list_names,metrics_list_values=metrics_list_values,identifiant='disc')

    if iteration % model_state_backup_int == 0:
        model_gan.save_weights(folder + graph_name+".h5")
        print("Weights saved at %s"%(folder + graph_name+".h5"))
        save_img(image,folder + graph_name+"_%d_img"%iteration)
        save_img(imageBruitee,folder + graph_name+"_%d_img_bruit"%iteration)
        print("Original images saved at %s"%(folder + graph_name+"_%d_img"%iteration))
        prediction_clean= model_gen.predict(image)
        prediction_clean *= 1/np.max(prediction_clean)
        prediction_noise = model_gen.predict(imageBruitee)
        prediction_noise *= 1/np.max(prediction_noise)
        save_img(prediction_clean,folder + graph_name+"_%d_img_fromclean"%iteration)
        save_img(prediction_noise,folder + graph_name+"_%d_img_fromnoise"%iteration)

        
        backup_prediction_path = folder + graph_name + "_step%d_predictions.txt"%(step)
        titles = ["Prediction_bruitee_orig_clean","Prediction_bruitee_orig_noise"]
        predictions = [str(oupt_disc_clean),str(oupt_disc_noise)]
        write_to_file(backup_file_path=backup_prediction_path,titles=titles,data_list_list=[predictions])

    backup_file_path = folder + graph_name+"_step%d_metrics.txt"%step
    titles = list(map(lambda x:str(x),metrics_list_names))
    data = list(map(lambda x:str(x),metrics_list_values))
    write_to_file(backup_file_path=backup_file_path,titles=titles,data_list_list=[data])
    print("".join([metrics_list_names[i] + " : " + str(metrics_list_values[i]) + "; " for i in range(len(metrics_list_values))]))

## Losses and metrics

In [0]:

def generate_functions(name,type_data,input_first_function,expression):
    global Lmetrics
    global Lnames
    Lnames.append("%s"%(expression).replace("y","rgb"))
    Lmetrics.append("metrics_%s_%s(%s)"%(type_data,name,",".join(input_first_function)))
    print("def metrics_%s_%s(%s):\n\tdef %s_%s(y_true,y_pred):\n\t\treturn %s\n\treturn %s_%s"%(type_data,name,",".join(input_first_function),type_data,name,expression,type_data,name))
    print()

In [0]:
Lmetrics = []
Lnames = []
type_metrique = ['probability'] #@param ["['probability']", "['rgb']", "['fft']", "['rgb']","['rgb','probability']","['rgb','fft']","['rgb','fft','probability']"] {type:"raw"}
for typeData in type_metrique:
    for analyse in ["max","mean","std"]:
        if typeData == "fft":
            for magnangle in ["magn","angle"]:
                generate_functions("abs_diff_%s_%s"%(magnangle,analyse),typeData,["r_%s"%magnangle,magnangle],"K.%s(K.abs(r_%s-%s))"%(analyse,magnangle,magnangle))
        else:
            i = "0" if typeData == "rgb" else "1"
            generate_functions("abs_diff_%s"%(analyse),typeData,[],"K.%s(K.abs(y_true-y_pred))"%(analyse))
print("[%s]"%(",".join(Lmetrics)))
print("[%s]"%(",".join(list(map(lambda x:"'"+x+"'",Lnames)))))

In [0]:
def metrics_rgb_abs_diff_max():
	def rgb_abs_diff_max(y_true,y_pred):
		return K.max(K.abs(y_true-y_pred))
	return rgb_abs_diff_max

def metrics_rgb_abs_diff_mean():
	def rgb_abs_diff_mean(y_true,y_pred):
		return K.mean(K.abs(y_true-y_pred))
	return rgb_abs_diff_mean

def metrics_rgb_abs_diff_std():
	def rgb_abs_diff_std(y_true,y_pred):
		return K.std(K.abs(y_true-y_pred))
	return rgb_abs_diff_std

# [metrics_rgb_abs_diff_max(),metrics_rgb_abs_diff_mean(),metrics_rgb_abs_diff_std()]
# ['K.max(K.abs(rgb_true-rgb_pred))','K.mean(K.abs(rgb_true-rgb_pred))','K.std(K.abs(rgb_true-rgb_pred))']

def metrics_probability_abs_diff_max():
	def probability_abs_diff_max(y_true,y_pred):
		return K.max(K.abs(y_true-y_pred))
	return probability_abs_diff_max

def metrics_probability_abs_diff_mean():
	def probability_abs_diff_mean(y_true,y_pred):
		return K.mean(K.abs(y_true-y_pred))
	return probability_abs_diff_mean

def metrics_probability_abs_diff_std():
	def probability_abs_diff_std(y_true,y_pred):
		return K.std(K.abs(y_true-y_pred))
	return probability_abs_diff_std

# [metrics_probability_abs_diff_max(),metrics_probability_abs_diff_mean(),metrics_probability_abs_diff_std()]
# ['K.max(K.abs(rgb_true-rgb_pred))','K.mean(K.abs(rgb_true-rgb_pred))','K.std(K.abs(rgb_true-rgb_pred))']

## Train

In [0]:
def loss_binary_MSE(inpt_gen_output,gen):
    def loss(y_true, y_pred):
        b_c = tf.keras.losses.binary_crossentropy(y_true[1], y_pred)
        MSE = K.mean(K.pow(gen- inpt_gen_output,2))
        return b_c+MSE*10**-2
    return loss

In [0]:
def train_progressive_model(index_model,taille_image,taille_batch,max_epoch=200, max_it=2000):
    train_data,validation_data,test_data = create_datasets()
    max_total_it = (max_step-1)*3*max_it
    tot_iteration = 0
    base_name = "Pretrain_fft_gan_%d"%index_model
    reprise_epoch = 50# @param {type:'number'}
    reprise_iteration = 0# @param {type:'number'}
    for epoch in range(max_epoch):
        object_dataset = Dataset(localisation_annexe_txt="./list_files.txt")
        graph_name = base_name
        graph_name = graph_name + "_%d"%(step)
        backup_path = graph_name + ".h5"
        model_gen,model_disc,model_gan,inpt_gen_output,gen = generate_models(name=graph_name)
        
        metrics_list_gen = [metrics_rgb_abs_diff_max(),metrics_rgb_abs_diff_mean(),metrics_rgb_abs_diff_std()]
        metrics_list_names_gen = ['MSE','K.max(K.abs(rgb_true-rgb_pred))','K.mean(K.abs(rgb_true-rgb_pred))','K.std(K.abs(rgb_true-rgb_pred))']

        metrics_list_disc = [metrics_probability_abs_diff_max(),metrics_probability_abs_diff_mean(),metrics_probability_abs_diff_std()]
        metrics_list_names_disc = ['Binary_crossentropy','K.max(K.abs(rgb_true-rgb_pred))','K.mean(K.abs(rgb_true-rgb_pred))','K.std(K.abs(rgb_true-rgb_pred))']
        adam_lr_neg_pow = 6#@param {type:'slider',min:0,max:10,step:1}
        adam_beta_1 = 0.99#@param {type:'slider',min:0.8,max:1.5,step:0.001}
        adam_beta_2 = 0.9405#@param {type:'slider',min:0.8,max:1.5,step:0.0001}
        opt = tf.keras.optimizers.Adam(learning_rate=10**-adam_lr_neg_pow, beta_1=adam_beta_1, beta_2=adam_beta_2, amsgrad=False)
        model_gen.compile(loss="MSE",metrics=metrics_list_gen,optimizer=opt)
        
        model_disc.compile(loss="binary_crossentropy",metrics=metrics_list_disc,optimizer=opt)
        model_gan.compile(loss=loss_binary_MSE(inpt_gen_output,gen),optimizer=opt)
        search_restore_pt(index_model,model_gan,base_name=graph_name)
        Ldisc,Lgan = [],[]
        for iteration in range(0 if step !=reprise_epoch else reprise_iteration,max_it):
            gen_input, gen_output,disc_input,disc_output,gan_input,gan_output = object_dataset.next_gan_batch(dataset='tr')
            metrics_list_values_gan = model_gan.train_on_batch([gen_input,gen_output],gan_output)
            loss_gan = metrics_list_values_gan
            
            metrics_list_values_disc = model_disc.train_on_batch(inpt_disc,outpt_disc)
            loss_disc = metrics_list_values_disc
            Ldisc.append(loss_disc[0])
            Lgan.append(loss_gan)
            compteur = 0
            max_repetitions_corrections = 100#@param {type:'number'}
            while abs(loss_disc[0]-loss_gan) > 0.5 and compteur < max_repetitions_corrections:
                image,imageBruitee = next_batch_bruit_example(train_data,iteration,max_it,taille_image,taille_batch)
                inpt_disc,outpt_disc,inpt_gen,outpt_gen,outpt_gan_disc = generate_inpts_outpts(image,imageBruitee)
                if loss_gan > loss_disc[0]:
                    metrics_list_values_gan = model_gan.train_on_batch([inpt_gen,outpt_gen],outpt_gan_disc)
                    loss_gan = metrics_list_values_gan

                    metrics_list_values_disc = model_disc.test_on_batch(inpt_disc,outpt_disc)
                    loss_disc = metrics_list_values_disc
                    print("Temporaire training GAN : Etape d'entrainement %d \t\titeration %d \tstatut %d \tpourcent, \terreur gan %.8e \t et disc %.8e"%(step, iteration,tot_iteration/max_total_it,loss_gan,loss_disc[0]))
                    Lgan.append(loss_gan)
                else:
                    metrics_list_values_disc = model_disc.train_on_batch(inpt_disc,outpt_disc)
                    loss_disc = metrics_list_values_disc

                    metrics_list_values_gan = model_gan.test_on_batch([inpt_gen,outpt_gen],outpt_gan_disc)
                    loss_gan = metrics_list_values_gan
                    print("Temporaire training Disc : Etape d'entrainement %d \t\titeration %d \tstatut %d \tpourcent, \terreur gan %.8e \t et disc %.8e"%(step, iteration,tot_iteration/max_total_it,loss_gan,loss_disc[0]))
                    Ldisc.append(loss_disc[0])
                compteur += 1
                plt.clf()
                plt.subplot(1,2,1)
                plt.plot(list(range(len(Ldisc))),Ldisc)
                plt.yscale("log")
                plt.subplot(1,2,2)
                plt.plot(list(range(len(Lgan))),Lgan)
                plt.yscale("log")
                plt.show()
            backup_fft(num_model=index_model,model_gen=model_gen,model_disc=model_disc,model_gan=model_gan,image=image,imageBruitee=imageBruitee,graph_name=graph_name,step=step,iteration=iteration,metrics_list_names_gen=metrics_list_names_gen,metrics_list_names_disc=metrics_list_names_disc,model_state_backup_int=5)
            
            print("Etape d'entrainement %d \t\titeration %d \tstatut %d \tpourcent, \terreur gan %.8e \t et disc %.8e"%(step, iteration,tot_iteration/max_total_it,loss_gan,loss_disc[0]))
            tot_iteration += 1

In [0]:
taille_image = 256# @param {type:'number'}
taille_batch = 5# @param {type:'number'}
numero_model = 12# @param {type:'number'}
#train_progressive_model(index_model=numero_model,taille_image=taille_image,taille_batch=taille_batch)

| lr | Beta 1 | Beta 2 |Erreur finale disc|
|:---:|:---:|:---:|:---:|
|$10^{-5}$|0.99|0.939|0.3|
|$10^{-6}$|0.99|0.944|4|
|$10^{-6}$|0.99|0.940|0.05 (non fini)|
|$10^{-6}$|0.99|0.941|4|
|$10^{-6}$|0.99|0.945||

In [0]:
# !kill -9 -1

In [0]:

base_name = "Pretrain_fft_gan_12"
graph_name = base_name
graph_name = graph_name + "_%d"%(50)
backup_path = graph_name + ".h5"
model_gen,model_disc,model_gan,inpt_gen_output,gen = generate_models(name=graph_name)

Building 129_Image


In [0]:
print(model_gan.layers)
print([l.name for l in model_gan.layers])
print(model_gan.outputs)

[<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fda8350bac8>, <tensorflow.python.keras.engine.training.Model object at 0x7fda8147f908>, <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fda806bb400>, <tensorflow.python.keras.engine.training.Model object at 0x7fda806bbfd0>]
['129_Image', 'gen', 'input_1', 'disc']
Tensor("disc/dense/BiasAdd:0", shape=(?, 1), dtype=float32)
